In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt

import fingernet as fnet

In [ ]:
# Get a single image
#img_path = "../datasets/NISTSD27/images/B101L9U.bmp"
img_path = "../datasets/NIST14/F0000001.bmp"
img = Image.open(img_path)
img.resize((256, 256))

In [ ]:
weights = "../models/released_version/Model.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"

model: fnet.FingerNetWrapper = fnet.get_fingernet(weights_path=weights, device=device)

to_tensor = lambda x: torch.from_numpy(np.array(x)).unsqueeze(0).unsqueeze(0).float().to(device) / 255.0
input_tensor = to_tensor(img)

In [ ]:
with torch.no_grad():
    fnet_output = model(input_tensor)

In [ ]:
for k, e in fnet_output.items():
    print(k, end=" ")
    if isinstance(e, torch.Tensor):
        fnet_output[k] = e.detach().cpu().numpy()
    elif isinstance(e, list):
        for i in range(len(e)):
            if isinstance(e[i], torch.Tensor):
                e[i] = e[i].detach().cpu().numpy()
        fnet_output[k] = e
fnet_output['input_path'] = img_path

In [ ]:
fnet.plot_output(fnet_output, figsize=(15, 5))